In [9]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [ ]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [10]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [37]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [20]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [99]:
def R2_test():
    print("Test-R2: Not elected validators should not earn reward")
    last_block = getLastBlockOfCurrentEpoch()
    current_block = getBlockNumber()
    print("last block of current epoch: ", last_block, " current block number: ", current_block)
    print("wait until the new epoch begins at block number ...", last_block + 1)
    num = int(input("enter the number of blocks you want to test: "))
    
    while current_block < last_block + 1:
        current_block = getBlockNumber()
    
    validator_infos = getAllValidatorInformation()
    not_elected = []
    rewards = dict()
    delegation_counts = dict()
    for i in validator_infos:
        if i['currently-in-committee'] == False:
            not_elected.append(i)
            address = i['validator']['address']
            delegations = i['validator']['delegations']
            amount = delegations[0]['amount']
            rewards[address] = amount
            delegation_counts[address] = len(delegations)

    last_block = getLastBlockOfCurrentEpoch()
    current_block = getBlockNumber()
    print("last block of current epoch: ", last_block, " current block number: ", current_block)
    print("wait until the last block of this epoch ...", last_block + 1)
    while current_block < last_block:
        current_block = getBlockNumber()
    
    # check the rewards
    iterations = 0
    failures = 0
    while iterations < num:
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            if i['currently-in-committee'] == False:
                address = i['validator']['address']
                delegations = i['validator']['delegations']
                amount = delegations[0]['amount']
                if address in rewards and rewards[address] != amount:
                    print("Error: reward not same for ", address, \
                          ", previous: ", rewards[address], ", new: ", amount)
                    failures = failures + 1
        iterations = iterations + 1
    if failures > 0:
        print("Test R2 failed")
    else:
        print("Test R2 succeeded")

In [100]:
R2_test()

Test-R2: Not elected validators should not earn reward
last block of current epoch:  31311  current block number:  31297
wait until the new epoch begins at block number ... 31312
enter the number of blocks you want to test: 1
last block of current epoch:  31349  current block number:  31312
wait until the last block of this epoch ... 31350
Error: reward not same for  one18j6h6tlsrymmmcucc7lajcdq9uh42gjv2szadf , previous:  1400000000000000000000 , new:  0
Test R2 failed
